# Definição do Problema

Classificação de documentos

# Configuração do ambiente de trabablho

- no container docker é instalado o tesseract-ocr
- instalando e importando bibliotecas

In [ ]:
import numpy as np 
import pandas as pd
import os

In [ ]:
# %pip install contractions

In [ ]:
# %pip install unidecode

In [ ]:
# %pip install yellowbrick

In [ ]:
# %pip install nltk

In [ ]:
# %pip install wordcloud

In [ ]:
# %pip install tensorflow

In [ ]:
# pytesseract: É uma biblioteca em Python que serve como uma interface para o Tesseract OCR Engine. O Tesseract é uma ferramenta de código aberto muito utilizada para realizar OCR em imagens.
# %pip install pytesseract

In [ ]:
# %pip install xgboost

In [ ]:
# %pip install lightgbm

In [ ]:
# %pip install catboost

In [ ]:
#----------SKLEARN--------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score # ,auc,roc_curve,precision_recall_curve
from sklearn.model_selection import train_test_split # , GridSearchCV, RandomizedSearchCV

# from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

# from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer,ClusteringScoreVisualizer
# from sklearn.metrics import silhouette_score

#-------NLTK--------------
# NLTK (Natural Language Toolkit) para processamento de linguagem natural (PLN)
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from unidecode import unidecode
from contractions import fix
# from nltk.util import ngrams

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.stem import WordNetLemmatizer #, LancasterStemmer
from wordcloud import WordCloud

#-------TENSORFLOW---------
import tensorflow as tf
from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import Dense, Input, SimpleRNN, LSTM , GRU, Bidirectional, Embedding
# from keras.layers import Dropout

# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# from keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')

from PIL import Image
import matplotlib.pyplot as plt
import pytesseract
import re
# from sklearn.linear_model import SGDClassifier
# from sklearn.svm import SVC
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier

# Coleta de Dados

- Dataset do kaggle: https://www.kaggle.com/datasets/ritvik1909/document-classification-dataset

- Baixar os dados e armazenar em "dataset/www-kaggle-com_datasets_ritvik1909_document-classification-dataset/"

# Análise Exploratória de Dados (EDA)

## Exemplo de e-mail

In [ ]:
path = "dataset/www-kaggle-com_datasets_ritvik1909_document-classification-dataset/email/doc_000042.png"
image = Image.open(path)
image

## Exemplo de Resumo

In [ ]:
path = "dataset/www-kaggle-com_datasets_ritvik1909_document-classification-dataset/resume/doc_000051.png"
image = Image.open(path)
image

## Exemplo de publicação cientifica

In [ ]:
path = "dataset/www-kaggle-com_datasets_ritvik1909_document-classification-dataset/scientific_publication/doc_000016.png"
image = Image.open(path)
image

# Extraindo dados de dentro da imagem via OCR

### Realizar OCR (Reconhecimento Óptico de Caracteres)

In [ ]:
# Extrai textos da imagem
text = pytesseract.image_to_string(image)
text

# Limpeza de Dados

### Lista de palavras que são consideradas stopwords (palavras vazias) para o idioma inglês. 

In [ ]:
# NLTK (Natural Language Toolkit) 
# conjunto de palavras stopwords para o idioma inglês 
stopwords_list = stopwords.words("english")
stopwords_list

# Preparação dos Dados para a modelagem

### Exemplo de processamento

Conversão para Minúsculas 

Substituição de Quebras de Linha e Tabulações 

Remoção de Espaços Extras 

Remoção de Números

Remoção de Pontuação 

Tokenização (word_tokenize(text)):
 - word_tokenize(text) utiliza a função word_tokenize do NLTK para dividir o texto em tokens individuais (palavras e pontuações). Isso é fundamental para trabalhar com o texto em nível de palavra.
Remoção de Pontuação Adicional e Stopwords

Lemmatização:
- A lematização é realizada usando o WordNetLemmatizer do NLTK. Cada palavra restante após a remoção de stopwords é lematizada, ou seja, reduzida à sua forma base (lema). Isso ajuda na normalização do texto, reduzindo variações morfológicas e simplificando o vocabulário.

Concatenação dos Tokens:
- todos os tokens lematizados são unidos em uma única string, onde cada token é separado por um espaço em branco. Esta string processada é então retornada como a saída da função.

In [ ]:
def preprocess_data(text):
    text = text.lower()
    text = text.replace("\n"," ").replace("\t"," ")
    text = re.sub("\s+"," ",text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    # tokens
    tokens = word_tokenize(text)
    
    data = [i for i in tokens if i not in punctuation]
    data = [i for i in data if i not in stopwords_list]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    final_text = []
    for i in data:
        word = lemmatizer.lemmatize(i)
        final_text.append(word)
        
    return " ".join(final_text)

preprocess_data(text)

In [ ]:
import os
# import pyarrow.parquet as pq

# Definição de Classes, opções de target
class_labels = {'email':0,'resume':1,'scientific_publication':2}

# usada para armazenar os textos extraídos das imagens após o pré-processamento
final_text = []
# usada para armazenar os rótulos numéricos correspondentes a cada documento
final_label = []
path = "dataset/www-kaggle-com_datasets_ritvik1909_document-classification-dataset"
image_folder = os.listdir(path)

# Verificar se o arquivo Parquet já existe, para não precisar reprocessar as imagem novamente
parquet_file = "document_texts.parquet"
if os.path.exists(parquet_file):
    # Carregar dados existentes se o arquivo Parquet já existir
    df = pd.read_parquet(parquet_file)
    final_text = df['Text'].tolist()
    final_label = df['Label'].tolist()
else:
    # para cada imagem do dataset
    for label_dir in image_folder:
        # caminho completo para o diretório atual de documentos (por classe 'email', 'resume' e 'scientific_publication')
        label_path = os.path.join(path,label_dir)
        # Itera sobre os arquivos dentro do diretório ""
        for j in os.listdir(label_path):
            # carrega a imagem.png
            image = Image.open(label_path+"/"+j)
            # Usa Tesseract OCR para extrair o texto da imagem
            text = pytesseract.image_to_string(image)
            # executa preprocessamento
            text_data = preprocess_data(text)
            # Adiciona o texto pré-processado (text_data) à lista final_text
            final_text.append(text_data)
            # Adiciona o rótulo numérico correspondente ao tipo de documento (label) à lista final_label, usando o dicionário class_labels para fazer a correspondência entre nome do diretório e rótulo numérico.
            final_label.append(class_labels[label_dir])
            
    # Criar um DataFrame com os resultados
    df = pd.DataFrame({'Text': final_text, 'Label': final_label})
    # Salvar DataFrame em arquivo Parquet
    df.to_parquet(parquet_file, index=False)
        

Criando Dataframe pandas com base nas lista 'final_text' e 'final_label'

Coluna 'Text' representa o texto

Coluna 'Label' tem valores 0, 1 e 2 (classes 'email', 'resume' e 'scientific_publication')


In [ ]:
df = pd.DataFrame()
df['Text'] = final_text
df['Label'] = final_label
df

In [ ]:
df.iloc[110]

In [ ]:
df['Text'][110]

In [ ]:
df['Label'][110]

# Analisar classes do DF

In [ ]:
df['Label'].value_counts()

In [ ]:
df['Label'].value_counts().plot(kind='pie',autopct="%.1f%%")
plt.title("Distribuição da quantidade de documentos")
plt.legend()
plt.show()

In [ ]:
sns.countplot(x=df['Label'])
plt.title("Document Distributions")
plt.show()

# Dividindo o dataframe em treino e teste (split)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df['Text'],df['Label'],shuffle=True,test_size=0.10)
print("Quantidade para treino: " + str(x_train.shape))
print("Quantidade para teste: " + str(x_test.shape))

In [ ]:
print(x_train.head(5))

In [ ]:
print(y_train.head(5))

Abordagem abaixo é útil para representar textos de forma numérica para serem usados como entrada em modelos de aprendizado de máquina, como classificadores ou modelos de clustering.

O TfidfVectorizer do scikit-learn é uma ferramenta que converte uma coleção de documentos de texto em uma matriz de características TF-IDF (Term Frequency-Inverse Document Frequency).  Matriz esparsa, que é eficiente para armazenar dados quando a maioria dos valores são zeros. A matriz esparsa economiza memória ao armazenar apenas os índices e valores dos elementos não-zero. A matriz densa ocupa mais memória porque armazena todos os elementos, incluindo os zeros.


- Criação de uma matriz TF-IDF para representar dados textuais. TF-IDF (Term Frequency-Inverse Document Frequency) é uma técnica para extrair características de texto que refletem a importância de uma palavra ou n-grama em relação a um documento em um corpus.

- TfidfVectorizer: Esta é uma classe do sklearn.feature_extraction.text que converte uma coleção de documentos de texto em uma matriz TF-IDF (Term Frequency-Inverse Document Frequency).

- ngram_range=(2,5): Define que serão considerados n-grams de tamanho mínimo 2 e máximo 5. Isso significa que o vetorizador criará características que são combinações de 2 até 5 palavras consecutivas.

- max_df=0.95: Remove termos que aparecem em mais de 95% dos documentos. Isso ajuda a remover palavras muito comuns que não são discriminativas.

- min_df=2: Remove termos que aparecem em menos de 2 documentos. Isso ajuda a remover palavras raras que podem não contribuir muito para o modelo.

- max_features=10000: Limita o número máximo de características (palavras ou n-grams) a serem extraídas. Aqui, será limitado a 10.000 características com base na frequência TF-IDF mais alta.


In [ ]:
# Configurando o TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(2,5), max_df=0.95, min_df=2 ,max_features=10000)

fit_transform(x_train): Este método ajusta o vetorizador ao conjunto de dados de treino x_train e, em seguida, transforma x_train em uma matriz TF-IDF. Isso significa que ele aprende o vocabulário do conjunto de treinamento e transforma cada documento (no caso, cada texto em x_train) em uma representação numérica usando TF-IDF.

A matriz resultante de tfidf.fit_transform(x_train) é uma matriz esparsa (tipicamente CSR - Compressed Sparse Row format), que é eficiente em termos de memória. Convertendo-a em uma matriz densa do NumPy.

In [ ]:
# Ajustando e transformando os dados de treinamento
tfidf_train = tfidf.fit_transform(x_train)

transform(x_test): transformando os dados de teste x_test em uma matriz TF-IDF usando o vocabulário aprendido durante o ajuste (fit_transform). É importante usar transform nos dados de teste, não fit_transform, porque queremos aplicar o mesmo vocabulário do conjunto de treinamento aos dados de teste.

In [ ]:
# Convertendo a matriz esparsa em uma matriz densa
tfidf_test = tfidf.transform(x_test)

In [ ]:
tfidf_train.A.shape

In [ ]:
# matriz esparsa de TF-IDF.
tfidf_train.A

In [ ]:
tfidf_train.A[0]

# Construindo o Modelo

Regressão Logistica

utilizando a classe LogisticRegression do scikit-learn para treinar um modelo de regressão logística com os dados de treinamento que foram transformados usando o vetorizador TF-IDF.

criando uma instância do modelo de Regressão Logística. A regressão logística é um algoritmo de aprendizado supervisionado usado para problemas de classificação binária e multiclasse.

In [ ]:
model = LogisticRegression()

# Treinamento do Modelo


tfidf_train.A é a matriz de características de treinamento transformada usando TF-IDF. O método .A converte a matriz esparsa para uma matriz densa, que é esperada pelo método fit da regressão logística.

y_train são os rótulos correspondentes aos exemplos de treinamento. Cada rótulo indica a classe à qual o exemplo pertence.

O vetorizador TF-IDF (tfidf) foi ajustado aos dados de treinamento (x_train) antes deste treinamento, definindo o vocabulário e calculando os pesos TF-IDF para cada palavra ou conjunto de palavras nos documentos.

tfidf_train.A é a matriz de características transformada usando TF-IDF. Ela contém representações numéricas das palavras presentes nos documentos de treinamento.

model.fit(tfidf_train.A, y_train) usa essas representações para treinar o modelo de regressão logística, onde tfidf_train.A são as características de entrada e y_train são os rótulos correspondentes às classes dos documentos de treinamento.

Após o treinamento, o modelo estará ajustado aos dados de treinamento e poderá ser usado para fazer previsões sobre novos dados (nesse caso, usando tfidf_test para transformar os dados de teste e model.predict para fazer previsões).

Este procedimento faz parte do fluxo típico de trabalho em aprendizado de máquina supervisionado para classificação de texto utilizando o método TF-IDF em conjunto com a regressão logística.

In [ ]:
model.fit(tfidf_train.A, y_train)

# Previsões

utilizando o modelo de regressão logística treinado (model) para fazer previsões sobre os dados de teste que foram transformados usando TF-IDF (tfidf_test).

model.predict() é o método utilizado para fazer previsões com o modelo treinado.

tfidf_test.A é a matriz de características de teste transformada usando TF-IDF. Assim como no treinamento, .A converte a matriz esparsa para uma matriz densa, que é esperada pelo método predict da regressão logística.


Após treinar o modelo de regressão logística com os dados de treinamento, agora está usando o modelo para prever as classes dos dados de teste.

tfidf_test.A contém as representações TF-IDF dos dados de teste, ou seja, como cada palavra nos documentos de teste foi transformada numericamente usando TF-IDF.

model.predict(tfidf_test.A) aplica o modelo treinado aos dados de teste para prever a classe de cada exemplo de teste.

O resultado das previsões é armazenado em y_pred, que é uma lista ou matriz (dependendo da forma de tfidf_test.A) contendo as classes previstas para os exemplos de teste.

y_pred contém as classes previstas para os exemplos de teste. Cada valor em y_pred corresponde à classe prevista para o exemplo correspondente em tfidf_test.A.

Com essas previsões, pode avaliar o desempenho do seu modelo comparando y_pred com os rótulos reais (y_test) dos dados de teste.

Este é o passo final em muitos fluxos de trabalho de aprendizado de máquina supervisionado, onde após treinar o modelo, fazemos previsões sobre novos dados para avaliar a eficácia do modelo treinado.

In [ ]:
# Previsões com dados de test
y_pred = model.predict(tfidf_test.A)
y_pred

### Previsões com dados de treinamento para detectar overfitting

utilizando o modelo de regressão logística treinado (model) para fazer previsões sobre os dados de treinamento que foram transformados usando TF-IDF (tfidf_train).

Após treinar o modelo de regressão logística com os dados de treinamento, está usando o modelo para prever as classes dos próprios dados de treinamento.

útil para verificar se o modelo está "decorando" os dados de treinamento, o que poderia indicar overfitting.

In [ ]:
y_pred_tr = model.predict(tfidf_train.A)
y_pred_tr

# Avaliação do modelo

In [ ]:
print("Matriz de confusão:\n",confusion_matrix(y_test,y_pred))
print()
print("Relatório de Classificação:\n",classification_report(y_test,y_pred))

In [ ]:
print("Confusion Matrix:\n",confusion_matrix(y_train,y_pred_tr))
print()
print("Classification Report:\n",classification_report(y_train,y_pred_tr))

In [ ]:
print(f"Accuracy on Test Data: {accuracy_score(y_test,y_pred)*100:.2f} %")
print(f"Accuracy on Train Data: {accuracy_score(y_train,y_pred_tr)*100:.2f} %")

# Regressão Logística

In [ ]:
lg_model = LogisticRegression(max_iter=2000,solver='liblinear',C=100)
lg_model.fit(tfidf_train.A,y_train)

In [ ]:
y_pred = lg_model.predict(tfidf_test.A)
y_pred

In [ ]:
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print()
print("Classification Report:\n",classification_report(y_test,y_pred))

In [ ]:
print(f"Accuracy on Test Data: {accuracy_score(y_test,y_pred)*100:.2f} %")

In [ ]:
y_test.values

# Arquivo Pickle para implantação

Sera gerado um arquivo model.pickle, new_model.pickle, tfidf.pickle

pickle é uma biblioteca do Python utilizada para serializar e desserializar objetos Python. Ela permite salvar objetos complexos em arquivos binários que podem ser facilmente carregados novamente posteriormente.

No contexto de aprendizado de máquina, é comum salvar modelos treinados e outros objetos importantes para que possam ser reutilizados sem a necessidade de treinamento novamente.

In [ ]:
import pickle
import json

Utilizando a biblioteca pickle do Python para salvar três objetos em arquivos binários, com o propósito de armazenar modelos que foram treinados e criados durante o processo de análise de dados ou aprendizado de máquina.

In [ ]:
class_labels

In [ ]:
# Class_labels
label_data = ['email', 'resume' ,'scientific_publication']

Salvando o Modelo de Regressão Logística (model)

model é o objeto que representa o modelo treinado de regressão logística.

O arquivo "model.pickle" conterá todos os parâmetros e configurações do modelo que foram aprendidos durante o treinamento.

In [ ]:
with open("model.pickle","wb") as file:
    pickle.dump(model,file)

Salvando o Objeto TF-IDF (tfidf)

tfidf é o objeto da classe TfidfVectorizer que foi utilizado para transformar os dados textuais em representações numéricas usando TF-IDF.

O arquivo "tfidf.pickle" contém todos os parâmetros e configurações do objeto tfidf, incluindo as palavras-chave aprendidas durante o ajuste.

In [ ]:
with open("tfidf.pickle","wb") as file1:
    pickle.dump(tfidf,file1)



Salvando um Novo Modelo (lg_model):

lg_model representa o modelo treinado, modelo de aprendizado de máquina de regressão logística

O arquivo "new_model.pickle" conterá todos os parâmetros e configurações do lg_model.

In [ ]:
with open("new_model.pickle","wb") as file2:
    pickle.dump(lg_model,file2)

# Implementação do Modelo

Implementar o modelo em um ambiente de produção, o que pode envolver a criação de APIs, integração com sistemas existentes, ou implementação em dashboards.

# Interpretação e Comunicação dos Resultados

Interpretar os resultados da análise ou modelo e comunicar as conclusões de maneira clara e acessível para os stakeholders, utilizando visualizações e relatórios.

# Monitoramento e Manutenção 

Monitorar o desempenho do modelo em produção e realizar manutenção contínua para assegurar que ele continue a funcionar corretamente e se adaptar a mudanças nos dados ou no ambiente.